In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
def generate_solution(C, T, R, fill_value=0, dtype=np.int16):
    """
    Create a (T, R) array with numbers from 1 to C appearing once each.
    If T*R > C, remaining elements are filled with `fill_value`.
    Numbers are randomly shuffled in the array.

    Parameters:
    - T (int): number of rows
    - R (int): number of columns
    - C (int): max unique number (from 1 to C)
    - fill_value (optional): value to fill remaining cells if any (default=0)
    - dtype (optional): numpy dtype for array (default=np.int16)

    Returns:
    - np.ndarray of shape (T, R)
    """
    total_size = T * R
    if total_size < C:
        raise ValueError(f"Array size {total_size} too small for {C} unique numbers")

    numbers = np.arange(1, C + 1, dtype=dtype)

    if total_size > C:
        filler = np.full(total_size - C, fill_value, dtype=dtype)
        numbers = np.concatenate((numbers, filler))

    np.random.shuffle(numbers)

    return numbers.reshape(T, R)


In [3]:
def generate_valid_solution(course_df, T, R, fill_value=0, dtype=np.int16):
    """
    Generate a valid solution using your 1D approach with transpose.
    Parameters:
    - course_df (pd.DataFrame): with 'course_id' and 'val' (duration)
    - T (int): rows
    - R (int): columns
    - fill_value (int): fill for empty slots

    Returns:
    - np.ndarray shape (T, R)
    """
    durations = course_df.set_index('course_id')['duration'].to_dict()
    course_ids = course_df['course_id'].tolist()

    total_slots = T * R
    required_slots = sum(durations.values())
    if required_slots > total_slots:
        raise ValueError("Not enough slots for all courses")

    arr_1d = np.full(total_slots, fill_value, dtype=dtype)

    # Find indices where 2-hour course can start:
    # These are all indices except those at the last column of each row in 1D form
    invalid_starts = {R * i + (R - 1) for i in range(T)}
    valid_starts = [i for i in range(total_slots) if i not in invalid_starts]

    # Compute valid starting indices for 2-hour courses
    valid_starts = [i for i in range(total_slots - 1) if (i + 1) % T != 0]

    # Place 2-hour courses first
    used_indices = set()
    for cid in course_ids:
        dur = durations[cid]
        if dur == 2:
            placed = False
            np.random.shuffle(valid_starts)
            for start_idx in valid_starts:
                # Ensure both start and next slot are unused
                if start_idx in used_indices or (start_idx + 1) in used_indices:
                    continue
                # Place the course at start_idx and start_idx + 1
                arr_1d[start_idx] = cid
                arr_1d[start_idx + 1] = cid
                used_indices.update({start_idx, start_idx + 1})
                placed = True
                break
            if not placed:
                raise RuntimeError(f"Cannot place 2-hour course {cid}")

    # Place 1-hour courses
    one_hour_courses = [cid for cid in course_ids if durations[cid] == 1]
    free_positions = [i for i in range(total_slots) if i not in used_indices]
    np.random.shuffle(one_hour_courses)

    np.random.shuffle(one_hour_courses)
    np.random.shuffle(free_positions)

    for cid, pos in zip(one_hour_courses, free_positions):
        arr_1d[pos] = cid
        used_indices.add(pos)

    # The rest remain fill_value

    # Reshape to (R, T), then transpose to (T, R)
    arr_2d = arr_1d.reshape(R, T).T

    return arr_2d


In [4]:
def array_to_multi_position_dict(arr, ignore_value=0):
    """
    Create a dictionary mapping each non-ignored value to a list of (row, col) positions
    where it appears in the 2D array.

    Parameters:
    - arr (np.ndarray): Input 2D array
    - ignore_value (int): Value to ignore (default=0)

    Returns:
    - dict: {value: [(row1, col1), (row2, col2), ...]}
    """
    from collections import defaultdict

    value_positions = defaultdict(list)

    for row in range(arr.shape[0]):
        for col in range(arr.shape[1]):
            val = arr[row, col]
            if val != ignore_value:
                value_positions[int(val)].append((row+1, col+1))

    return dict(sorted(value_positions.items()))


In [5]:
def export_to_txt(arr, folder="solutions", filename="solution.txt"):
    # Ensure the folder exists
    os.makedirs(folder, exist_ok=True)

    # Full file path
    filepath = os.path.join(folder, filename)

    # Convert array to string without clipping or wrapping
    with np.printoptions(threshold=np.inf, linewidth=10000):
        arr_str = np.array2string(arr, separator=', ')

    # Write to file
    with open(filepath, 'w') as f:
        f.write(arr_str)

In [6]:
def import_from_txt(folder="output", filename="output.txt"):
    filepath = os.path.join(folder, filename)

    with open(filepath, 'r') as f:
        content = f.read()

    # Convert string back to numpy array
    arr = np.array(eval(content), dtype=np.int16)  # Use eval because it's safe with known input like this
    return arr

In [7]:
def get_course_duration(df, course_id):
    row = df[df['course_id'] == course_id]
    if row.empty:
        raise ValueError(f"Course ID '{course_id}' not found in DataFrame.")
    return int(row.iloc[0]['duration'])


In [8]:
def is_solution_valid(arr, criteria_functions, course_df, ignore_value=0):
    """
    Validate a solution array against multiple criteria.

    Parameters:
    - arr (np.ndarray): 2D solution array.
    - criteria_functions (list of callables): Functions accepting (solution_dict, course_df) and returning bool.
    - course_df (pd.DataFrame): DataFrame with course info.
    - ignore_value (int): Value in arr to ignore (default=0).

    Returns:
    - bool: True if all criteria pass, False otherwise.
    """
    solution_dict = array_to_multi_position_dict(arr, ignore_value)

    for check in criteria_functions:
        if not check(solution_dict, course_df):
            return False
    return True


In [ ]:
def time_slot_criteria(solution_dict, course_df):
    """
    Validate that each course in solution_dict appears the correct number of times
    according to course_df, and that 2-hour courses occupy two consecutive slots
    in the same row.

    Parameters:
    - solution_dict (dict): Mapping from course_id to list of (row, col) positions.
    - course_df (pd.DataFrame): DataFrame with 'course_id' and 'val' columns.

    Returns:
    - bool: True if all courses satisfy their duration and consecutive slot criteria.
    """
    for course_id, positions in solution_dict.items():
        expected_count = get_course_duration(course_df, course_id)
        
        # Check course duration count
        if len(positions) != expected_count:
            print(f"Failed on first test: {course_id}")
            return False
        
        # Additional check for 2-hour courses
        if expected_count == 2:
            (r1, c1), (r2, c2) = positions
            if c1 != c2 or abs(r1 - r2) != 1:
                print(f"Failed on second test: {course_id}")
                return False

    return True


In [10]:
fmipa = pd.read_csv('csv/fmipa.csv')[["course_id", "duration", "time_slot", "room_id"]]
fmipa.head()

,course_id,duration,time_slot,room_id
0,1,2,A132,26
1,2,2,C092,20
2,3,2,A132,31
3,4,2,D132,1
4,5,2,A132,36


In [11]:
arrays = {col: fmipa[col].to_numpy() for col in fmipa.columns}

course_id = arrays["course_id"]
duration = arrays["duration"]
time_slot = arrays["time_slot"]
room_id = arrays["room_id"]

In [12]:
C = 51
T = 55
R = 54

# Generate solution
for i in range(10):
    export_to_txt(generate_valid_solution(fmipa, T, R), "solutions", f"solution_{i+1}.txt")

In [13]:
sol = generate_valid_solution(fmipa, T, R)
export_to_txt(sol)

In [14]:
# Check if solution is valid
arr = import_from_txt("solutions", "solution_3.txt")

criteria_list = [
    time_slot_criteria,
]

is_valid = is_solution_valid(arr, criteria_list, course_df=fmipa)
print("Is solution valid?", is_valid)

Is solution valid? True
